<a href="https://colab.research.google.com/github/ejdogar/NLP/blob/main/Text_Generation_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import keras

In [ ]:
docs = ["Hey how are you",
        "Hey whats up",
        "Pakistan lost the match",
        "it is raining",
        "Charles Leclerc will be the F1 champion of 2024",
        "Ronaldo is the best football player",
        "Schumacher was the best driver ever",
        "Lewis Hamilton is a cheat",
        "Sebastian Vettel is struggling"
        ]

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(docs)

In [ ]:
sequences = tokenizer.texts_to_sequences(docs)

In [ ]:
print(sequences)

[[3, 5, 6, 7], [3, 8, 9], [10, 11, 1, 12], [13, 2, 14], [15, 16, 17, 18, 1, 19, 20, 21, 22], [23, 2, 1, 4, 24, 25], [26, 27, 1, 4, 28, 29], [30, 31, 2, 32, 33], [34, 35, 2, 36]]


In [ ]:
word2int = tokenizer.word_index
print(word2int)
vocab_size = len(word2int)
vocab_size

{'the': 1, 'is': 2, 'hey': 3, 'best': 4, 'how': 5, 'are': 6, 'you': 7, 'whats': 8, 'up': 9, 'pakistan': 10, 'lost': 11, 'match': 12, 'it': 13, 'raining': 14, 'charles': 15, 'leclerc': 16, 'will': 17, 'be': 18, 'f1': 19, 'champion': 20, 'of': 21, '2024': 22, 'ronaldo': 23, 'football': 24, 'player': 25, 'schumacher': 26, 'was': 27, 'driver': 28, 'ever': 29, 'lewis': 30, 'hamilton': 31, 'a': 32, 'cheat': 33, 'sebastian': 34, 'vettel': 35, 'struggling': 36}


36

In [ ]:
int2word = {idx+1: word for idx, word in enumerate(word2int)}

In [ ]:
from keras.utils import pad_sequences
sequences = pad_sequences(sequences, padding="post")
sequences

array([[ 3,  5,  6,  7,  0,  0,  0,  0,  0],
       [ 3,  8,  9,  0,  0,  0,  0,  0,  0],
       [10, 11,  1, 12,  0,  0,  0,  0,  0],
       [13,  2, 14,  0,  0,  0,  0,  0,  0],
       [15, 16, 17, 18,  1, 19, 20, 21, 22],
       [23,  2,  1,  4, 24, 25,  0,  0,  0],
       [26, 27,  1,  4, 28, 29,  0,  0,  0],
       [30, 31,  2, 32, 33,  0,  0,  0,  0],
       [34, 35,  2, 36,  0,  0,  0,  0,  0]], dtype=int32)

In [ ]:
maxlength = len(sequences[0])
maxlength

9

In [ ]:
input_seq = []
target_seq = []
for i in range(len(sequences)):
  input_seq.append(sequences[i][:-1])
  target_seq.append(sequences[i][1:])

print(f"{input_seq[0]=}")
print(f"{target_seq[0]=}")

input_seq[0]=array([3, 5, 6, 7, 0, 0, 0, 0], dtype=int32)
target_seq[0]=array([5, 6, 7, 0, 0, 0, 0, 0], dtype=int32)


In [ ]:
seq_len = maxlength - 1
def one_hot_encoder(sequence, batch_size, seq_len, vocab_size):
  feature = np.zeros((batch_size, seq_len, vocab_size+1), dtype = np.float32)

  for i in range(batch_size):
    for k in range(seq_len):
      feature[i, k, sequence[i][k]] = 1

  return feature

In [ ]:
batch_size = len(docs)
input_seq = one_hot_encoder(input_seq, batch_size, seq_len, vocab_size)

In [ ]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [ ]:
is_cuda = torch.cuda.is_available()

if is_cuda:
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("GPU not available, running on CPU...")

GPU not available, running on CPU...


In [ ]:
class Model(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, n_layers):
    super(Model, self).__init__()

    self.hidden_dim = hidden_dim
    self.n_layers = n_layers

    self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first = True)
    self.fc = nn.Linear(hidden_dim, output_size)


  def forward(self, x):
    batch_size = x.size(0)

    hidden = self.init_hidden(batch_size)

    out, hidden = self.rnn(x, hidden)

    out = out.contiguous().view(-1, self.hidden_dim)

    out = self.fc(out)

    return out, hidden


  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
    return hidden

In [ ]:
model = Model(input_size = vocab_size+1, output_size = vocab_size+1, hidden_dim = 8, n_layers = 1)
model = model.to(device)

In [ ]:
n_epochs = 7000
lr = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [ ]:
input_seq = input_seq.to(device)
print(f"{input_seq.shape=}")
for epoch in range(1, n_epochs+1):
  optimizer.zero_grad()

  out, hidden = model(input_seq)
  out.to(device)
  target_seq.to(device)
  loss = criterion(out, target_seq.view(-1).long())
  loss.backward()
  optimizer.step()

  if epoch%50  == 0:
    print('Epoch: {}/{}......'.format(epoch, n_epochs), end=' ')
    print("Loss : {:.4f}".format(loss.item()))

input_seq.shape=torch.Size([9, 8, 37])
Epoch: 50/7000...... Loss : 3.7481
Epoch: 100/7000...... Loss : 3.7205
Epoch: 150/7000...... Loss : 3.6937
Epoch: 200/7000...... Loss : 3.6670
Epoch: 250/7000...... Loss : 3.6392
Epoch: 300/7000...... Loss : 3.6096
Epoch: 350/7000...... Loss : 3.5770
Epoch: 400/7000...... Loss : 3.5406
Epoch: 450/7000...... Loss : 3.4996
Epoch: 500/7000...... Loss : 3.4533
Epoch: 550/7000...... Loss : 3.4014
Epoch: 600/7000...... Loss : 3.3436
Epoch: 650/7000...... Loss : 3.2803
Epoch: 700/7000...... Loss : 3.2117
Epoch: 750/7000...... Loss : 3.1383
Epoch: 800/7000...... Loss : 3.0608
Epoch: 850/7000...... Loss : 2.9806
Epoch: 900/7000...... Loss : 2.8997
Epoch: 950/7000...... Loss : 2.8206
Epoch: 1000/7000...... Loss : 2.7453
Epoch: 1050/7000...... Loss : 2.6752
Epoch: 1100/7000...... Loss : 2.6109
Epoch: 1150/7000...... Loss : 2.5526
Epoch: 1200/7000...... Loss : 2.4999
Epoch: 1250/7000...... Loss : 2.4525
Epoch: 1300/7000...... Loss : 2.4099
Epoch: 1350/7000...

In [ ]:
def predict(model, words):
  sentence = np.array([[word2int[word] for word in words]])
  sentence = one_hot_encoder(sentence, 1, sentence.shape[1], vocab_size)
  sentence = torch.from_numpy(sentence)
  sentence = sentence.to(device)

  out, hidden = model(sentence)

  prob = nn.functional.softmax(out[-1], dim=0).data
  word_ind = torch.max(prob, dim=0)[1].item()

  return int2word[word_ind+1], hidden

In [ ]:
def sample(model, out_size, start = "hey"):
  model.eval()

  start = start.lower()

  start = start.split()

  words = [word for word in start]

  size = out_size - len(words)

  print("===============",size)
  for i in range(size):
    o, h = predict(model, words)
    words.append(o)

  return " ".join(words)

In [ ]:
sample(model, 7, "is")

=============== 6


'is is is will f1 a sebastian'